In [2]:
using ParallelTemperingMonteCarlo,BenchmarkTools,DelimitedFiles,MachineLearningPotential

Completed the MLP module based on the librunner.so file written by A. Knoll. Time to use this in PTMC simulations so I can finally generate results. This requires the librunnerjulia file and use of the MLP package.

The following functions are taken from the final stages of development of the MLP. We have not yet established the calculation of the f_matrix or organised the struct containing the neural network.

In [3]:
function energy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
    n_atoms = length(positions)
    g_mat = total_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
    return g_mat,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
end
function threnergy_calculation(positions,dist2_mat,f_mat,total_symmetry_vector,neuralnetwork)
    n_atoms = length(positions)
    g_mat = total_thr_symm_calc(positions,dist2_mat,f_mat,total_symmetry_vector)
    return g_mat,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
end
function energy_calculation(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector,neuralnetwork)
    n_atoms = length(positions)
    g_mat = total_symm!(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector)
    return g_matrix,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
end

function threnergy_calculation(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector,neuralnetwork)
    n_atoms = length(positions)
    g_mat = total_thr_symm!(g_matrix, positions,newpos,dist2_mat,new_dis_vec,f_mat,new_f_vec,atomindex,total_symmetry_vector)
    return g_matrix,sum(forward_pass(g_mat,n_atoms,neuralnetwork))
end

threnergy_calculation (generic function with 2 methods)

First: establishing the struct.

This requires the symmetry functions, the nnp values itself, the g and f matrices used in the calculation and a dummy vector new_f_vec to preassign memory for later. I'd prefer to point to the nnp and symmetryfunction structs rather than save copies, I'll look into this later. 

In [4]:
mutable struct  RuNNerPotential <: AbstractMachineLearningPotential
    nnp::NeuralNetworkPotential
    symmetryfunctions::Vector{AbstractSymmFunction}
    g_matrix::Array
    f_matrix::Array
    new_f_vec::Vector
end

In [6]:
function RuNNerPotential(nnp,symmetryfunctions,n_atoms)
    g_matrix = zeros(length(symmetryfunctions),n_atoms)
    f_matrix = zeros(length(symmetryfunctions),n_atoms)
    new_f_vec = zeros(n_atoms)
    return RuNNerPotential(nnp,symmetryfunctions,g_matrix,f_matrix,new_f_vec)
end

RuNNerPotential